In [22]:
import json
import requests
import psycopg2
from psycopg2.extensions import AsIs
import pandas as pd
from time import sleep
from datetime import datetime, timedelta
from config import vk_access_token
from sqldb import GetDataPostsSQL, sendDataContentSQL, sendDataPostsSQL,getIndexesFromMainDB


In [23]:

def getVkJson(list_of_indexes) -> pd.DataFrame:
    df = pd.DataFrame()
    for x in list_of_indexes:
        url = 'https://api.vk.com/method/'
        method_wall = 'wall.get'
        v = '5.131'
        params_wall = {
        'owner_id':x[0]*-1,
        'access_token':vk_access_token,
        'count': 30,
        'v':v
    }
        req_wall = requests.get(url + method_wall, params = params_wall, allow_redirects=False)
        req_wall.close
        sleep(0.5)
        df = pd.concat([df, pd.DataFrame(req_wall.json()['response']['items'])])

    return df.reset_index()


def dateStamper(arr):
    return pd.to_datetime(datetime.utcfromtimestamp(arr).strftime('%Y-%m-%d %H:%M:%S'))

def currentDate():
    return pd.to_datetime(datetime.now().strftime('%Y-%m-%d %H:%M:%S'))


def changeSignOwnerID(arr):
    return arr *-1


def getPhotoMetaData(arr) -> dict:
    if 'photo' in arr[0].values():
        return arr[0]['photo']


def getAttachments(arr) -> dict:
    if 'photo' in arr[0].values():
        return arr[0]['photo']['sizes']

def getPhotoID(arr) -> dict:
    if 'post_id' in arr.keys() :
        return arr['post_id']


def getSizes(arr) -> pd.DataFrame:
    temp_list = []
    for ind in range(len(arr)):
        for x in arr['meta_photo'][ind]['sizes']:
            if 'x' in x.values():
                temp_list.append( x['url'])

    return pd.DataFrame(temp_list, columns=['url'])


def convert_asis(arr):
    return AsIs(arr)


def dateSTAMP(arr):
    return datetime.now().strftime('%Y-%m-%d %H:%M:%S')

def takeNewestPosts(sql_db, cur_db) -> pd.DataFrame:
    temp_df = pd.DataFrame()
    for x in cur_db['content_id'].to_list():
        if x not in sql_db:
            temp_df = pd.concat([temp_df, cur_db.loc[cur_db['content_id'] == x]])
    return temp_df



In [24]:
main_list = getIndexesFromMainDB()
df_from_vk = getVkJson(main_list)

[INFO] the connection is closed, Func getIndexesFromMainDB has finished


In [25]:
def mainColumns(func) -> pd.DataFrame:
    def inner(*args, **kwargs):
        result = func(*args, **kwargs)
        return result[['id','owner_id','text','date','attachments']]
    return inner

In [26]:
@mainColumns
def requiredData(df) -> pd.DataFrame:
    return df.loc[(df['carousel_offset'] != 0.0)&(df['marked_as_ads'] != 1)&(df['attachments'] != None)]


In [27]:
frame_main_df = requiredData(df_from_vk)[:]

In [28]:
def getdicts(func):
    def get0(df):
        result = func(df)
        try:
            return result.get('sizes')
        except:
            return None        
    return get0

In [29]:
def gettingUrl(func):
    def inner(*args, **kwargs):
        result = func(*args, **kwargs)
        try:
            for x in result:
                if  x.get('type') == 'x':
                    return x['url']
        except:
            return None
    return inner
    

In [30]:
@gettingUrl
@getdicts
def unpaker(df):
    try:
        return df[0].get('photo')
    except:
        return None

    

In [31]:
frame_main_df['raw_data'] = frame_main_df['attachments'].apply(unpaker)


In [32]:
frame_main_df

,id,owner_id,text,date,attachments,raw_data
1,12179012,-460389,Приyчил cвою дeвушку,1652199600,"[{'type': 'video', 'video': {'access_key': 'ea...",None
2,12178958,-460389,,1652197847,"[{'type': 'photo', 'photo': {'album_id': -7, '...",https://sun9-42.userapi.com/s/v1/if2/V4t8vLnzx...
3,12178894,-460389,На концeрте Metallica фaнатка родилa cына пoд ...,1652196000,"[{'type': 'photo', 'photo': {'album_id': -7, '...",https://sun9-60.userapi.com/s/v1/if2/Mh7P-34Mz...
4,12178823,-460389,,1652194107,"[{'type': 'photo', 'photo': {'album_id': -7, '...",https://sun9-42.userapi.com/s/v1/if2/QyrxXXyEC...
5,12178734,-460389,,1652192319,"[{'type': 'photo', 'photo': {'album_id': -7, '...",https://sun9-73.userapi.com/s/v1/if2/WquSkmhsM...
...,...,...,...,...,...,...
219,126351,-127661835,Ромaнтики поcт,1652117520,"[{'type': 'photo', 'photo': {'album_id': -7, '...",https://sun9-80.userapi.com/s/v1/if2/kQkSGc8LH...
223,126319,-127661835,Лисухa-борцухa,1652095712,"[{'type': 'photo', 'photo': {'album_id': -7, '...",https://sun9-83.userapi.com/s/v1/if2/rs8Jx7ZNR...
224,126309,-127661835,Драмa с элeментами хоррорa,1652090563,"[{'type': 'photo', 'photo': {'album_id': -7, '...",https://sun9-80.userapi.com/s/v1/if2/-jxKCiIkx...
237,126098,-127661835,,1651913790,"[{'type': 'photo', 'photo': {'album_id': -7, '...",https://sun9-28.userapi.com/s/v1/if2/cPCVVo7Dj...
